In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Load the dataset
file_path = r"C:\Users\hassa\Downloads\SQL\PROJECT\final_book_dataset_kaggle2.csv"
books_df = pd.read_csv(file_path)

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Load the DataFrame into the SQLite database
books_df.to_sql('books', conn, index=False, if_exists='replace')


830

In [3]:
# 1. Average price of books
query_1 = """
SELECT ROUND (AVG(CAST(price AS REAL)), 2) AS average_price
FROM books
"""
average_price = pd.read_sql_query(query_1, conn)
print("Average Price of Books:", average_price.iloc[0, 0])

Average Price of Books: 46.49


In [4]:
# 2. Authors with the highest average review ratings
query_2 = """
SELECT author, AVG(CAST(avg_reviews AS REAL)) AS avg_review_rating
FROM books
WHERE author IS NOT NULL
GROUP BY author
ORDER BY avg_review_rating DESC
LIMIT 10
"""
highest_rated_authors = pd.read_sql_query(query_2, conn)
print("Highest Rated Authors:")
print(highest_rated_authors)

Highest Rated Authors:
                                              author  avg_review_rating
0                [ Xudong Ma, Vishakh Hegde, et al.]                5.0
1                                  [ Tonny J. Oyana]                5.0
2                                    [ Toby Culshaw]                5.0
3                                  [ Steven F. Lott]                5.0
4  [ Stephen P. Borgatti, Martin G. Everett, et al.]                5.0
5        [ Stephen J. Wright,  and , Benjamin Recht]                5.0
6                                   [ Soledad Galli]                5.0
7                                [ Sinchan Banerjee]                5.0
8                         [ Sherwin John C. Tragura]                5.0
9                                 [ Santos Ozoemena]                5.0


In [5]:
# 3. Most common publishers for data science books
query_3 = """
SELECT publisher, COUNT(*) AS count
FROM books
GROUP BY publisher
ORDER BY count DESC
LIMIT 10
"""
common_publishers = pd.read_sql_query(query_3, conn)
print("Most Common Publishers:")
print(common_publishers)

Most Common Publishers:
                                           publisher  count
0                                               None    116
1                Packt Publishing (October 31, 2022)      3
2                Packt Publishing (January 21, 2022)      3
3                  Packt Publishing (April 29, 2022)      3
4            Wiley; 1st edition (September 21, 2022)      2
5  Packt Publishing - ebooks Account (January 10,...      2
6                  Packt Publishing (March 19, 2021)      2
7                   Packt Publishing (June 30, 2022)      2
8         O'Reilly Media; 1st edition (May 11, 2021)      2
9    O'Reilly Media; 1st edition (December 20, 2022)      2


In [6]:
# 4. Correlation between number of pages and average review rating
query_4 = """
SELECT pages, avg_reviews
FROM books
WHERE pages IS NOT NULL AND avg_reviews IS NOT NULL
"""
pages_reviews_df = pd.read_sql_query(query_4, conn)
pages_reviews_df['pages'] = pd.to_numeric(pages_reviews_df['pages'], errors='coerce')
pages_reviews_df['avg_reviews'] = pd.to_numeric(pages_reviews_df['avg_reviews'], errors='coerce')
pages_reviews_df = pages_reviews_df.dropna()
correlation = pages_reviews_df.corr().loc['pages', 'avg_reviews']
print("Correlation between Number of Pages and Average Review Rating:", correlation)

Correlation between Number of Pages and Average Review Rating: 0.046571471060341894


In [7]:
# 5. Compare new and used book prices
books_df[['new_price', 'used_price']] = books_df['price (including used books)'].str.split('-', expand=True)
books_df['new_price'] = books_df['new_price'].str.replace('[\$,]', '', regex=True).astype(float)
books_df['used_price'] = books_df['used_price'].str.replace('[\$,]', '', regex=True).astype(float)
books_df.to_sql('books', conn, index=False, if_exists='replace')

query_5 = """
SELECT AVG(new_price) AS average_new_price, AVG(used_price) AS average_used_price
FROM books
"""
new_vs_used_prices = pd.read_sql_query(query_5, conn)
print("Average New Price:", new_vs_used_prices.iloc[0, 0])
print("Average Used Price:", new_vs_used_prices.iloc[0, 1])

Average New Price: 39.4581994459836
Average Used Price: 57.27444444444438


In [8]:
# 6. Top 10 highest-rated books
query_6 = """
SELECT title, author, avg_reviews
FROM books
ORDER BY CAST(avg_reviews AS REAL) DESC
LIMIT 10
"""
top_rated_books = pd.read_sql_query(query_6, conn)
print("Top 10 Highest-Rated Books:")
print(top_rated_books)

Top 10 Highest-Rated Books:
                                               title  \
0        Topological Data Analysis with Applications   
1  Python Programming for Beginners: Python Progr...   
2  Microsoft 365 Excel: The Only App That Matters...   
3                  Analyzing Social Networks Using R   
4  Machine Learning Explained The Simple Way: A B...   
5  Talent Intelligence: Use Business and People D...   
6  Fundamentals of Music Processing: Using Python...   
7  Data-Driven Organization Design: Delivering Pe...   
8  Statistical Analysis of fMRI Data, second edit...   
9  Time Series for Data Science: Analysis and For...   

                                              author  avg_reviews  
0  [ Gunnar Carlsson,  and , Mikael Vejdemo-Johan...          5.0  
1                                [ Donald P. Wilson]          5.0  
2                                     [ Mike Girvin]          5.0  
3  [ Stephen P. Borgatti, Martin G. Everett, et al.]          5.0  
4              

In [ ]:
# 7. Distribution of book lengths (in pages)
query_7 = """
SELECT pages
FROM books
WHERE pages IS NOT NULL
"""
book_lengths = pd.read_sql_query(query_7, conn)
plt.figure(figsize=(10, 6))
plt.hist(book_lengths['pages'], bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Number of Pages')
plt.ylabel('Number of Books')
plt.title('Distribution of Book Lengths (in Pages)')
plt.grid(True)
plt.show()

In [9]:
# 8. Distribution of books by language
query_8 = """
SELECT language, COUNT(*) AS count
FROM books
GROUP BY language
ORDER BY count DESC
"""
language_distribution = pd.read_sql_query(query_8, conn)
print("Books by Language:")
print(language_distribution)

Books by Language:
                                             language  count
0                                             English    745
1                                                None     71
2                                             Spanish      4
3      • How to make better business decisions using       1
4                     you will discover all you need       1
5                                    standard library      1
6   Unqualified, Japanese (Dolby Digital 2.0 Mono)...      1
7   This Python programming guide assumes certain ...      1
8        Scroll to the top of the page and click the       1
9                    English (Dolby Digital 2.0 Mono)      1
10  English (DTS-HD Master Audio 5.1), French (DTS...      1
11  Concepts are presented in a "to-the-point" sty...      1
12                                       "Brilliant."      1


In [10]:
# 9. Books with the highest number of reviews
query_9 = """
SELECT title, author, n_reviews
FROM books
ORDER BY CAST(REPLACE(n_reviews, ',', '') AS INTEGER) DESC
LIMIT 10
"""
most_reviewed_books = pd.read_sql_query(query_9, conn)
print("Most Reviewed Books:")
print(most_reviewed_books)


Most Reviewed Books:
                                               title  \
0  Start with Why: How Great Leaders Inspire Ever...   
1  Monty Python and the Holy Grail (Special Edition)   
2  Invisible Women: Data Bias in a World Designed...   
3  Cracking the Coding Interview: 189 Programming...   
4  Python Crash Course, 2nd Edition: A Hands-On, ...   
5  Tikaton Reptile Heat Pad - Adjustable Temperat...   
6  Principles for Dealing with the Changing World...   
7  REPTI ZOO Reptile Terrarium Thermometer Hygrom...   
8  Python: - The Bible- 3 Manuscripts in 1 book: ...   
9  Monty Python's Life Of Brian - The Immaculate ...   

                     author n_reviews  
0                      None    26,560  
1                      None    10,103  
2  [ Caroline Criado Perez]     8,819  
3                      None     7,953  
4           [ Eric Matthes]     7,767  
5                      None     6,045  
6              [ Ray Dalio]     5,517  
7                      None     4,388  
8 

In [11]:
# 10. Price range for highest-rated books
query_10 = """
SELECT title, author, price, `price (including used books)`, avg_reviews
FROM books
WHERE avg_reviews = '5.0'
"""
highest_rated_books_prices = pd.read_sql_query(query_10, conn)
print("Highest Rated Books' Price Ranges:")
print(highest_rated_books_prices)

Highest Rated Books' Price Ranges:
                                                title  \
0         Topological Data Analysis with Applications   
1   Python Programming for Beginners: Python Progr...   
2   Microsoft 365 Excel: The Only App That Matters...   
3                   Analyzing Social Networks Using R   
4   Machine Learning Explained The Simple Way: A B...   
..                                                ...   
56  R for Political Data Science (Chapman & Hall/C...   
57  Targeted Learning in Data Science: Causal Infe...   
58                                   Integrating Data   
59            Introduction to Biomedical Data Science   
60                                  Game Data Science   

                                               author  price  \
0   [ Gunnar Carlsson,  and , Mikael Vejdemo-Johan...  54.19   
1                                 [ Donald P. Wilson]    NaN   
2                                      [ Mike Girvin]  34.95   
3   [ Stephen P. Borgatt